In [1]:
## Define Comands
commands = [
 #   "_background_noise_",
    "down",
    "left",
    "right",
    "stop"
]

4-element Vector{String}:
 "down"
 "left"
 "right"
 "stop"

In [2]:
## Read the test data set
using BSON: @load, @save
using Flux

@load "../rundir/test_ds_x.bson" test_ds_x
@load "../rundir/test_ds_y.bson" test_ds_y

In [3]:
## Load the Flux model
using Random
@load "../models/tf-inspired-model-KWS4-ACC0p96.bson" model
testmode!(model,true)
ŷ = Flux.onecold(model(test_ds_x))
y = Flux.onecold(test_ds_y)
test_acc = sum(y .== ŷ) ./ length(y)
println("Test Accuracy: $(test_acc * 100)")


Test Accuracy: 96.18461538461538


In [4]:
model

Chain(
  Upsample(:bilinear, size = (32, 32)),
  Conv((3, 3), 1 => 32, relu),          # 320 parameters
  BatchNorm(32),                        # 64 parameters, plus 64
  Conv((3, 3), 32 => 64, relu),         # 18_496 parameters
  BatchNorm(64),                        # 128 parameters, plus 128
  MaxPool((2, 2)),
  Dropout(0.25),
  Flux.flatten,
  Dense(12544 => 128, relu),            # 1_605_760 parameters
  Dropout(0.5),
  Dense(128 => 4),                      # 516 parameters
)         # Total: 12 trainable arrays, 1_625_284 parameters,
          # plus 4 non-trainable, 192 parameters, summarysize 6.202 MiB.

In [5]:
model(test_ds_x[:,:,:,1:1]) |> display
test_ds_y[:,1]

4×1 Matrix{Float32}:
 -31.911196
  -2.257361
  26.68097
 -28.481256

4-element OneHotVector(::UInt32) with eltype Bool:
 ⋅
 ⋅
 1
 ⋅

In [6]:
## Load/Transfer parameters
# using NPZ
params = Dict()
params[1] = Dict()
params[1]["weight"] = model[2].weight
params[1]["bias"] = model[2].bias
params[2] = Dict()
params[2]["mu"] = model[3].μ
params[2]["sigmasq"] = model[3].σ²
params[2]["beta"] = model[3].β
params[2]["gamma"] = model[3].γ
params[3] = Dict()
params[3]["weight"] = model[4].weight
params[3]["bias"] = model[4].bias
params[4] = Dict()
params[4]["mu"] = model[5].μ
params[4]["sigmasq"] = model[5].σ²
params[4]["beta"] = model[5].β
params[4]["gamma"] = model[5].γ


64-element Vector{Float32}:
 0.8480308
 1.1235139
 0.9318675
 0.9300036
 0.70785654
 0.7001765
 0.66839767
 0.85313547
 0.82983357
 0.86777425
 0.9391121
 0.71013194
 0.88868445
 ⋮
 0.7070167
 1.2329556
 1.0360843
 0.93945825
 0.7674355
 0.99972916
 0.69751155
 1.087317
 1.1903495
 0.9152731
 0.67870754
 1.1445674

In [7]:
## Preprocess with model
acts = Flux.activations(model,test_ds_x[:,:,:,1:3])
acts[1]

32×32×1×3 Array{Float32, 4}:
[:, :, 1, 1] =
 0.000169161  0.00691752  0.00433719  0.00309809  …  7.45125f-5   7.13244f-5
 0.00150886   0.00669999  0.0020485   0.00561728     0.000140923  0.000141315
 8.04043f-5   0.00534018  0.00423471  0.00360951     0.000115724  0.000121904
 0.00285406   0.00601265  0.00430659  0.0083117      8.38959f-5   5.18783f-5
 0.00243114   0.00484808  0.00415893  0.00716842     8.78499f-5   2.35298f-5
 0.000936706  0.00269159  0.00115331  0.00434005  …  0.000119399  2.13426f-5
 0.00103799   0.0139712   0.0030597   0.00468179     6.66317f-5   6.1384f-5
 0.00199391   0.00346311  0.00696605  0.00257653     7.57161f-5   2.98204f-5
 0.000486208  0.011191    0.0354752   0.00735334     0.000246431  0.000148811
 0.195649     4.3505      1.65302     0.525253       0.000132661  5.5094f-5
 0.398431     2.41272     4.3603      1.50374     …  0.00335448   0.00201374
 0.511134     4.92554     8.24178     5.14543        0.00246714   0.00155139
 0.283527     2.15433     4.672

In [8]:
acts[6][:,:,:,1]

14×14×64 Array{Float32, 3}:
[:, :, 1] =
  0.113844    0.00188104   0.0400826  …   0.0794623   0.0437855    0.073096
  0.101655    0.0806421    0.0319573      0.101586    0.075988     0.0790743
  3.71769     0.765875     0.150307      -0.0421657   0.00831313   0.032723
  5.14971     6.91656     -0.32768       -0.150648    0.73677     -0.300608
 -0.56545    -0.56545     -0.56545        0.83667     0.859522    -0.56545
 -0.56545    -0.56545     -0.56545    …   0.911555   -0.56545     -0.56545
 -0.56545    -0.274306     2.44692       -0.104646    0.0620036   -0.500802
  1.55817     0.481112     4.8534         4.99743     2.30395      0.344133
  0.0389335  -0.0834568    1.54018        1.62427    -0.155903    -0.56545
  1.29232     1.0426       0.78924        1.31742    -0.548737    -0.56545
  0.276992    0.477075     0.628226   …   1.75941     0.847095    -0.119086
  0.0666657   0.285937     0.330477       0.50742     0.299375     0.0684733
  0.0913925   0.0406391    0.107841       0.163159

In [9]:
# Numpy Inference
using PyCall
pushfirst!(pyimport("sys")."path", "../py")
inf = pyimport("inference")
out = inf._cnn_infer(acts[1][:,:,:,1],params)


14×14×64 Array{Float32, 3}:
[:, :, 1] =
  0.113844    0.00188088   0.0400827  …   0.0794624   0.0437856    0.0730961
  0.101655    0.0806423    0.0319571      0.101586    0.0759879    0.0790742
  3.71769     0.765875     0.150307      -0.0421656   0.00831297   0.032723
  5.14971     6.91656     -0.327681      -0.150648    0.73677     -0.300608
 -0.56545    -0.56545     -0.56545        0.83667     0.859523    -0.56545
 -0.56545    -0.56545     -0.56545    …   0.911554   -0.56545     -0.56545
 -0.56545    -0.274305     2.44692       -0.104645    0.0620037   -0.500802
  1.55817     0.481112     4.8534         4.99743     2.30395      0.344133
  0.0389336  -0.0834568    1.54018        1.62426    -0.155903    -0.56545
  1.29232     1.0426       0.78924        1.31742    -0.548737    -0.56545
  0.276992    0.477074     0.628226   …   1.75941     0.847095    -0.119086
  0.0666657   0.285937     0.330477       0.50742     0.299375     0.0684735
  0.0913927   0.0406391    0.107841       0.16315

In [12]:
MSE = sum((out .- acts[6][:,:,:,1]).^2) / prod(size(out))

4.390535f-13